# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
# Import the findspark module to help with initializing Spark in Jupyter or other environments
import findspark
findspark.init()

# Import the SparkSession class from the pyspark.sql module
from pyspark.sql import SparkSession

# Create a SparkSession
ss = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.some.config.option", "config-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-30 17:51:18,665 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
# Copy the local file 'clickstream.csv' to the Hadoop Distributed File System (HDFS) root directory
! hadoop fs -copyFromLocal clickstream.csv /

# List the contents of the HDFS root directory, specifically checking for 'clickstream.csv'
! hadoop fs -ls -h /clickstream.csv


copyFromLocal: `/clickstream.csv': File exists
-rw-r--r--   1 jovyan supergroup     30.7 M 2024-10-30 17:49 /clickstream.csv


In [5]:
# Read a CSV file named 'clickstream.csv' into a DataFrame
df = ss.read.csv("file:///home/jovyan/work/clickstream.csv", sep="\t", header=True)

# Display the first 100 rows of the DataFrame
df.show(30)

# Create a temporary view named 'clickstream' that allows for SQL queries to be run on this DataFrame
df.createOrReplaceTempView("clickstream")


+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

In [34]:
main_df = ss.sql("""
WITH step1 AS (
    SELECT user_id, session_id, event_page, timestamp,
           SUM(CASE WHEN LOWER(event_type) LIKE '%error%' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS is_error
    FROM clickstream
),
step2 AS (
    SELECT user_id, session_id, event_page, timestamp,
           LEAD(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS next_page
    FROM step1
    WHERE is_error = 0
),
step3 AS (
    SELECT user_id, session_id,
           CONCAT_WS('-', COLLECT_LIST(event_page)) AS route
    FROM step2
    WHERE event_page != next_page OR next_page IS NULL
    GROUP BY user_id, session_id
)
SELECT route, COUNT(*) AS route_count 
FROM step3 
GROUP BY route 
ORDER BY route_count DESC 
LIMIT 10;
""")

In [35]:
main_df.show()

+-------------------+-----------+
|              route|route_count|
+-------------------+-----------+
|               main|       8184|
|       main-archive|       1112|
|        main-rabota|       1047|
|      main-internet|        897|
|         main-bonus|        870|
|          main-news|        769|
|       main-tariffs|        677|
|        main-online|        587|
|         main-vklad|        517|
|main-rabota-archive|        170|
+-------------------+-----------+



In [48]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Load the clickstream data into a DataFrame
clickstream_df = ss.read.csv("file:///home/jovyan/work/clickstream.csv", header=True, inferSchema=True, 
                                sep="\t")

In [49]:
step1 = (clickstream_df
         .withColumn("is_error", 
                     F.sum(F.when(F.lower(F.col("event_type")).like('%error%'), 1).otherwise(0))
                     .over(Window.partitionBy("user_id", "session_id").orderBy("timestamp"))))

In [50]:
window = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

step2 = (step1 
         .filter(step1.is_error == 0)
         .withColumn("prev_page", F.lag("event_page").over(window))
)

In [51]:
step3 = (step2
         .filter((step2.event_page != step2.prev_page) | step2.prev_page.isNull())
         .groupBy("user_id", "session_id")
         .agg(F.concat_ws('-', F.collect_list("event_page")).alias("route"))
)

In [52]:
main_df = (step3
            .groupBy("route")
            .agg(F.count("*").alias("count"))
            .orderBy(F.col("count").desc())
            .limit(10)
)
main_df.show()

+-------------------+-----+
|              route|count|
+-------------------+-----+
|               main| 8184|
|       main-archive| 1112|
|        main-rabota| 1047|
|      main-internet|  897|
|         main-bonus|  870|
|          main-news|  769|
|       main-tariffs|  676|
|        main-online|  587|
|         main-vklad|  518|
|main-rabota-archive|  170|
+-------------------+-----+



In [62]:
import json

In [60]:
routes_dict = {row['route']: row['count'] for row in top_routes}

# output JSON file
output_json_path = "top10_routes.json"
with open(output_json_path, 'w') as json_file:
    json.dump(routes_dict, json_file, indent=4)

In [61]:
result

['{"route":"main","count":8184}',
 '{"route":"main-archive","count":1112}',
 '{"route":"main-rabota","count":1047}',
 '{"route":"main-internet","count":897}',
 '{"route":"main-bonus","count":870}',
 '{"route":"main-news","count":769}',
 '{"route":"main-tariffs","count":676}',
 '{"route":"main-online","count":587}',
 '{"route":"main-vklad","count":518}',
 '{"route":"main-rabota-archive","count":170}']

In [65]:
with open("/home/jovyan/work/result.json", "w") as f:
    json.dump(result, f)

In [ ]:
!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/nbobylev/w6/1